In [6]:
# !git add getStackOverFlowDataToSql.ipynb
# !git reset getStackOverFlowDataToSql.ipynb
# !git checkout -b Ibrahim_branch
# !git add NaturalEarthToAzureSQL.ipynb
# !git commit -m "Added script for collecting and updating Geographical data"
# !git push origin Ibrahim_branch


# !git checkout Ibrahim_branch  
# !git pull origin Ibrahim_branch  
# !git checkout Ibrahim_branch
# !git merge Ibrahim_branch  


To https://dev.azure.com/AmaliTech-BI/StackOverflow/_git/StackOverflow
   6968758..68d8d0a  Ibrahim_branch -> Ibrahim_branch


In [89]:
import os
import requests
import zipfile
import geopandas as gpd
import pyodbc  
import pandas as pd
import ogr
import shutil

# from airflow import DAG
# from airflow.hooks.base import BaseHook
# from airflow.providers.odbc.hooks.odbc import OdbcHook
# from airflow.operators.python import PythonOperator
from datetime import datetime
import csv,sys

# from airflow.models import BaseOperator
from sqlalchemy import create_engine,text,String  
# from airflow.utils.dates import days_ago
# from airflow.providers.mssql.hooks.mssql import MsSqlHook
import pyodbc
# from airflow.providers.microsoft.mssql.hooks.mssql import MsSqlHook


In [98]:
from sqlalchemy import create_engine, text, MetaData, Unicode, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
import os
import pandas as pd
import geopandas as gpd
from io import BytesIO
import requests
import zipfile
from sqlalchemy import inspect  

#  connection parameters
server_name = ""  
database_name = ""  
username = ""  
password = ""  



# Create the connection string
connection_string = f"mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server&charset=utf8mb4"

# Create an SQL Server engine
engine = create_engine(connection_string, connect_args={'convert_unicode': True})

# Define the language columns
language_columns = ['NAME_AR']

def convert_shape_to_csv(gdf):
    csv_buffer = BytesIO()
    gdf.to_csv(csv_buffer, index=False, encoding='utf-8-sig', mode='wb')
    csv_buffer.seek(0)
    return csv_buffer

def compare_table_structure(engine, table_name, df):
    inspector = inspect(engine)  # Use the inspect function
    
    existing_columns = inspector.get_columns(table_name)  # Get existing columns
    
    reflected_columns = [col['name'] for col in existing_columns]
    df_columns = df.columns
    
    return set(reflected_columns) == set(df_columns)

def transfer_geographical_data_to_sql():
    file_urls = [
        "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_1_states_provinces.zip",
        "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_populated_places.zip",
        "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_countries_arg.zip"
    ]
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    
    arabic_name_column = "NAME_AR"
    
    for file_url in file_urls:
        response = requests.get(file_url, headers=headers, stream=True)
        if response.status_code == 200:
            with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
                extracted_folder = os.path.splitext(os.path.basename(file_url))[0]
                zip_ref.extractall(extracted_folder)
            
            shapefile_path = os.path.join(extracted_folder, f"{os.path.splitext(os.path.basename(file_url))[0]}.shp")
            
            gdf = gpd.read_file(shapefile_path)
            csv_buffer = convert_shape_to_csv(gdf)
            csv_content = csv_buffer.getvalue()
            df = pd.read_csv(BytesIO(csv_content), encoding='utf-8-sig')
                # Define data types for columns
            def set_data_type(column):
                if column in language_columns:
                    return Unicode(length='max', collation='SQL_Latin1_General_CP1_CI_AS')
                elif isinstance(df[column].dtype, sqlalchemy.types.TypeEngine):
                    return df[column].dtype
                else:
                    return String(collation='SQL_Latin1_General_CP1_CI_AS')
            
            table_name = os.path.splitext(os.path.basename(file_url))[0]
            
            with engine.connect() as conn:
                if not compare_table_structure(engine, table_name, df):
                    print(f"Table structure for '{table_name}' does not match. Skipping update.")
                    continue
                
                conn.execute(text(f"DROP TABLE IF EXISTS {table_name}"))
                
            dtype = {col: set_data_type(col) for col in df.columns}
            
            df.to_sql(table_name, engine, if_exists='replace', index=False,dtype=dtype)
            print(f"Table '{table_name}' updated or replaced.")

# Call the function
transfer_geographical_data_to_sql()


Table structure for 'ne_10m_admin_1_states_provinces' does not match. Skipping update.


C:\Users\IbrahimOsuman\AppData\Local\Temp\ipykernel_6020\2561155753.py:72: DtypeWarning: Columns (104,105,106,108,110,114,115,116,118,119,122,123,124,125,126,127,128,129,131,132,133,135,136) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(BytesIO(csv_content), encoding='utf-8-sig')


Table structure for 'ne_10m_populated_places' does not match. Skipping update.
Table structure for 'ne_10m_admin_0_countries_arg' does not match. Skipping update.


In [88]:
# from sqlalchemy import create_engine, String
# from sqlalchemy.types import Unicode
# from sqlalchemy.types import TypeEngine
# from io import BytesIO
# import sqlalchemy
# import pymysql

# def convert_shape_to_csv(gdf):
#     csv_buffer = BytesIO()
#     gdf.to_csv(csv_buffer, index=False, encoding='utf-8-sig', mode='wb')
#     csv_buffer.seek(0)
#     return csv_buffer

# def transfer_geographical_data_to_sql():
# #     file_urls = [
# #         "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_1_states_provinces.zip",
# #         "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_populated_places.zip",
# #         "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_countries_arg.zip"
# #     ]
#     file_urls = [
#         "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_countries_arg.zip",
#     ]
    
    
    
    
#     language_columns = ['NAME_AR', ]
    
# #     # Define your connection parameters
# #     server_name = "data-science.ck0dwzfc16xf.eu-west-1.rds.amazonaws.com"  # Replace with your server name
# #     database_name = "TrendStreamDB"  # Replace with your database name
# #     username = "admin"  # Replace with your username
# #     password = "dAta$ci3nCe"  # Replace with your password

    
    
#     headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    
#     arabic_name_column ="NAME_AR"
    
#     # Define data types for columns
#     def set_data_type(column):
#         if column in language_columns:
#             return Unicode(length='max', collation='SQL_Latin1_General_CP1_CI_AS')
#         elif isinstance(df[column].dtype, sqlalchemy.types.TypeEngine):
#             return df[column].dtype
#         else:
#             return String(collation='SQL_Latin1_General_CP1_CI_AS')

#     for file_url in file_urls:
#         response = requests.get(file_url, headers=headers, stream=True)
#         if response.status_code == 200:
#             with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
#                 extracted_folder = os.path.splitext(os.path.basename(file_url))[0]
#                 zip_ref.extractall(extracted_folder)
            
#             # Get the shapefile path from the extracted folder
#             shapefile_path = os.path.join(extracted_folder, f"{os.path.splitext(os.path.basename(file_url))[0]}.shp")
            
#             # Convert the shapefile to GeoDataFrame
#             gdf = gpd.read_file(shapefile_path)
            
#             # Convert the GeoDataFrame to CSV
#             csv_buffer = convert_shape_to_csv(gdf)
            
#             # Read and print the first few lines of the CSV content
#             csv_content = csv_buffer.getvalue()
#             df = pd.read_csv(BytesIO(csv_content), encoding='utf-8-sig')
#             print(f"First few lines of CSV content:\n{df.head()}")
            
#             # Save user data to SQL Server
#             server_name = "localhost\\MYSQL2019"
#             database_name = "Data"
#             username = "connect"
#             password = "O4i4m9b8g##"
            
#             # Create the connection string
#             connection_string = f"mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server&charset=utf8mb4"

#             # Create an SQL Server engine
#             engine = create_engine(connection_string,connect_args={'convert_unicode': True})
            
#             # Get the table name from the file URL
#             table_name = os.path.splitext(os.path.basename(file_url))[0]
            
#             # Drop the table if it already exists
#             drop_table_query = text(f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name}")

#             with engine.connect() as conn:
#                 conn.execute(drop_table_query)
                
#             # Define data types with collation for specified columns
            
# #             dtype = {
# #                     col: Unicode(length='max', collation='SQL_Latin1_General_CP1_CI_AS') 
# #                     if col in language_columns 
# #                     else String(collation='SQL_Latin1_General_CP1_CI_AS') 
# #                     for col in df.columns
# #                 }
# #             print(dtype)
# #             dtype = {col: set_data_type(col) for col in df.columns}
# #             dtype = df.apply(lambda col: String(collation='SQL_Latin1_General_CP1_CI_AS') if col.dtype == 'object' else col.dtype)
            
#             # Save the DataFrame to the SQL Server table
#             df.to_sql(table_name, engine, if_exists='replace', index=False)

# # Call the function
# transfer_geographical_data_to_sql()


First few lines of CSV content:
  ADM0_A3_AR       featurecla  scalerank  LABELRANK SOVEREIGNT SOV_A3  \
0        IDN  Admin-0 country          0          2  Indonesia    IDN   
1        MYS  Admin-0 country          0          3   Malaysia    MYS   
2        CHL  Admin-0 country          0          2      Chile    CHL   
3        BOL  Admin-0 country          0          3    Bolivia    BOL   
4        PER  Admin-0 country          0          2       Peru    PER   

   ADM0_DIF  LEVEL               TYPE  TLC  ... FCLASS_TR FCLASS_ID  \
0         0      2  Sovereign country  1.0  ...       NaN       NaN   
1         0      2  Sovereign country  1.0  ...       NaN       NaN   
2         0      2  Sovereign country  1.0  ...       NaN       NaN   
3         0      2  Sovereign country  1.0  ...       NaN       NaN   
4         0      2  Sovereign country  1.0  ...       NaN       NaN   

   FCLASS_PL FCLASS_GR FCLASS_IT  FCLASS_NL FCLASS_SE FCLASS_BD  FCLASS_UA  \
0        NaN       NaN  